In [7]:
import subprocess
def get_html_from_url(url):
    cmds = ["/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome", "--headless", "--dump-dom"]
    cmds = ["/Applications/Google Chrome.app/Contents/MacOS/Google Chrome", "--headless", "--dump-dom"]
    cmds.append(url)
    result = subprocess.run(cmds, stdout=subprocess.PIPE)
    text = result.stdout.decode('utf-8')
    return text
    
# t = get_html_from_url("https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:systems-of-equations/x2f8bb11595b61c86:introduction-to-systems-of-equations/v/trolls-tolls-and-systems-of-equation")
# t
# # result = subprocess.run(['pwd'], stdout=subprocess.PIPE)
# # text = result.stdout.decode('utf-8')
# # text


In [82]:
# unit_url = "https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:systems-of-equations/x2f8bb11595b61c86:introduction-to-systems-of-equations"
unit_url = "https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:inequalities-systems-graphs"
site = "https://www.khanacademy.org"

In [83]:
from xml.dom.minidom import parseString

html = get_html_from_url(unit_url)



[0522/131928.313956:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0522/131928.460477:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.


In [84]:
from html.parser import HTMLParser
class Parser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.href = ''
        self.urls = []

    def handle_starttag(self, tag, attrs):
        if (tag == 'a'):
            for (k,v) in attrs:
                if (k == 'href'):
                    self.href = v
        elif (tag == 'span'):
            for (k,v) in attrs:
                if (k == 'aria-label' and v == 'Video'):
                    self.urls.append(self.href)            



parser = Parser()
parser.feed(html)
parser.urls


['/math/algebra/x2f8bb11595b61c86:inequalities-systems-graphs/x2f8bb11595b61c86:checking-solutions-of-two-variable-inequalities/v/graphing-inequalities-1',
 '/math/algebra/x2f8bb11595b61c86:inequalities-systems-graphs/x2f8bb11595b61c86:checking-solutions-of-two-variable-inequalities/v/testing-solutions-for-a-system-of-inequalities',
 '/math/algebra/x2f8bb11595b61c86:inequalities-systems-graphs/x2f8bb11595b61c86:graphing-two-variable-inequalities/v/graphing-inequalities',
 '/math/algebra/x2f8bb11595b61c86:inequalities-systems-graphs/x2f8bb11595b61c86:graphing-two-variable-inequalities/v/graphing-linear-inequalities-in-two-variables-example-2',
 '/math/algebra/x2f8bb11595b61c86:inequalities-systems-graphs/x2f8bb11595b61c86:graphing-two-variable-inequalities/v/graphing-linear-inequalities-in-two-variables-3',
 '/math/algebra/x2f8bb11595b61c86:inequalities-systems-graphs/x2f8bb11595b61c86:graphing-two-variable-inequalities/v/graphical-system-of-inequalities',
 '/math/algebra/x2f8bb11595b61

In [32]:
video_url = site + parser.urls[0]
video_html = get_html_from_url(video_url)




[0521/115618.386214:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0521/115618.527860:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.


In [49]:
class VideoParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.start_h1 = False
        self.start_div = False
        self.start_iframe = False
        
        self.name = ''
        self.desc = ''

    def handle_starttag(self, tag, attrs):
        if (tag == 'h1'):
            self.start_h1 = True       
        elif (tag == 'div'):
            for (k,v) in attrs:
                if (k == 'id' and v == 'ka-videoPageTabs-tabbedpanel-content'):
                    self.start_div = True
        elif (tag == 'iframe'):
            print(attrs)
            for (k,v) in attrs:
                if(k == 'data-youtubeid'):
                    print(v)

    def handle_endtag(self, tag):
        if (tag == 'h1'):
            self.start_h1 = False
        elif (tag == 'div'):
            self.start_div = False
            
    def handle_data(self, data):
        if self.start_h1:
            self.name = data.title()
        if self.start_div:
            self.desc += data

video_parser = VideoParser()
video_parser.feed(video_html)
print(video_parser.name, video_parser.desc)




[('src', 'https://www.googletagmanager.com/ns.html?id=GTM-WSX5PBP'), ('height', '0'), ('width', '0'), ('style', 'display:none;visibility:hidden')]
Systems Of Equations: Trolls, Tolls (1 Of 2) A troll forces us to use algebra to figure out the make-up of his currency. We end up setting up a system of equations. Created by Sal Khan.


In [78]:
def get_searched_value(video_html, searchStr):
    x = video_html.find(searchStr)
    start = video_html.find('"', x+len(searchStr))
    end = video_html.find('"', start+1)
    return video_html[start+1:end]


def get_topic(video_html):
    desc =  get_searched_value(video_html, '"translatedDescription":')
    name = get_searched_value(video_html, '"translatedTitle":').title()
    videoID = get_searched_value(video_html, '"translatedYoutubeId":')
    return {
        "name": name,
        "description": desc,
        "texts": ["text1.md"],
        "video_parts": [{
            "id": videoID
        }],
        "node_type": "lecture",
        "parent_id": "khan-algebra-1",
        "thumbnail": ""
    }

In [86]:

topics = []
for url in parser.urls:
    video_url = site + url
    video_html = get_html_from_url(video_url)
    topic = get_topic(video_html)
    print(json.dumps(topic, indent=4))
    topics.append(topic)
    

topics
    
    
    
    

[0522/132141.910254:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0522/132142.043472:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.


{
    "name": "Testing Solutions To Inequalities ",
    "description": "Sal checks if the ordered pairs (3,5) and (1,-7) are solutions of the inequality 5x-3y\u226525.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "2oGsLdAWxlk"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}


[0522/132207.598824:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0522/132207.634762:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.


{
    "name": "Testing Solutions To Systems Of Inequalities",
    "description": "Sal checks whether the ordered pair (2,5) is a solution of the following system: y\u22652x+1 and x>1.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "XzYNh2wpO0A"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}


[0522/132224.010125:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0522/132224.046031:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.


...

[0522/132635.626455:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0522/132635.661431:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.


{
    "name": "Graphs Of Two-Variable Inequalities Word Problem",
    "description": "Given the graph of a two-variable linear inequality that models a context about dog biscuits, Sal finds if the dog can get enough biscuits.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "DhiiGFuUE9I"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}


[0522/132647.194982:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0522/132647.229774:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.


{
    "name": "Creativity Break: What Can We Do To Expand Our Creative Skills?",
    "description": "Experts share their advice on how to expand our creative skills.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "SFUG5KpIf_Q"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}


[{'name': 'Testing Solutions To Inequalities ',
  'description': 'Sal checks if the ordered pairs (3,5) and (1,-7) are solutions of the inequality 5x-3y≥25.',
  'texts': ['text1.md'],
  'video_parts': [{'id': '2oGsLdAWxlk'}],
  'node_type': 'lecture',
  'parent_id': 'khan-algebra-1',
  'thumbnail': ''},
 {'name': 'Testing Solutions To Systems Of Inequalities',
  'description': 'Sal checks whether the ordered pair (2,5) is a solution of the following system: y≥2x+1 and x>1.',
  'texts': ['text1.md'],
  'video_parts': [{'id': 'XzYNh2wpO0A'}],
  'node_type': 'lecture',
  'parent_id': 'khan-algebra-1',
  'thumbnail': ''},
 {'name': 'Intro To Graphing Two-Variable Inequalities',
  'description': 'Learn how to graph two-variable linear inequalities like y≤4x+3.',
  'texts': ['text1.md'],
  'video_parts': [{'id': 'unSBFwK881s'}],
  'node_type': 'lecture',
  'parent_id': 'khan-algebra-1',
  'thumbnail': ''},
 {'name': 'Stylesheet',
  'description': 'stylesheet',
  'texts': ['text1.md'],
  'vid